<a href="https://colab.research.google.com/github/Devininthelab/Toy-Repo/blob/main/LoRA4_HuggingFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Import packages

In [1]:
!pip3 install loralib

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 2. Data Loader

In [3]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

BATCH_SIZE = 256

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

100%|██████████| 170498071/170498071 [00:18<00:00, 9205475.21it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


# 3. Modeling

In [4]:
def trainable_params(model):
    total_params = 0
    for param in model.parameters():
        if param.requires_grad:
            total_params += param.numel()

    print(f"Trainable params: {total_params/1e6:.2f} M")

## 3.1: Load pretrained wweight from ImageNet1k



In [5]:
from torchvision.models import vgg16, VGG16_Weights
vgg16_model = vgg16(weights=VGG16_Weights.IMAGENET1K_V1)

vgg16_classifier_ckpt = vgg16_model.classifier.state_dict()

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:05<00:00, 92.6MB/s]


In [6]:
trainable_params(vgg16_model)

Trainable params: 138.36 M


In [7]:
trainable_params(vgg16_model.classifier)

Trainable params: 123.64 M


## 3.2: Add a layer with outpt features is 10 for Cifa10

In [8]:
new_classififer = nn.Sequential(
    *vgg16_model.classifier,
    nn.ReLU(inplace=True),
    nn.Dropout(p=0.5, inplace=False),
    nn.Linear(in_features=1000, out_features=10, bias=True)
)

In [9]:
new_classififer

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
  (7): ReLU(inplace=True)
  (8): Dropout(p=0.5, inplace=False)
  (9): Linear(in_features=1000, out_features=10, bias=True)
)

In [10]:
trainable_params(new_classififer)

Trainable params: 123.65 M


## 3.3: Apply LoRA to all Linear layer

In [12]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 22.4 MB/s eta 0:00:00


In [13]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["0", "3", "6", "9"], # layer_indices for applying lora
    lora_dropout=0.1,
    bias="all"
)

lora_classifier = get_peft_model(new_classififer, config)


In [14]:
trainable_params(lora_classifier)

Trainable params: 0.70 M


## 3.4: Wrap pre-trained VGG16 features (will be frozen) with LoRA clf

In [15]:
class CLS_model(nn.Module):
    def __init__(self, features, classifier)->None:
        super().__init__()
        self.features = features
        for param in self.features.parameters():
            param.requires_grad_(False)
        self.avgpool = nn.AdaptiveAvgPool2d((7, 7))
        self.classifier = classifier

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x


In [17]:
model = CLS_model(
    features = vgg16_model.features.eval(),
    classifier = lora_classifier
)
model.to(device)

CLS_model(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dila

In [18]:
trainable_params(model)


Trainable params: 0.70 M


# 4. Training

In [19]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

In [20]:
model.train()

start = time.time()
for epoch in range(10):
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(trainloader, 0):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch + 1}/{10}], Average Loss: {running_loss / len(trainloader):.4f}, GPU used: {torch.cuda.memory_allocated(0)/1e9:.2f} G')

print('Finished Training')
print(f'Training time: {time.time() - start:.2f} s')

Epoch [1/10], Average Loss: 1.5645, GPU used: 0.59 G
Epoch [2/10], Average Loss: 1.1726, GPU used: 0.59 G
Epoch [3/10], Average Loss: 1.0853, GPU used: 0.59 G
Epoch [4/10], Average Loss: 1.0497, GPU used: 0.59 G
Epoch [5/10], Average Loss: 1.0203, GPU used: 0.59 G
Epoch [6/10], Average Loss: 1.0041, GPU used: 0.59 G
Epoch [7/10], Average Loss: 0.9846, GPU used: 0.59 G
Epoch [8/10], Average Loss: 0.9733, GPU used: 0.59 G
Epoch [9/10], Average Loss: 0.9607, GPU used: 0.59 G
Epoch [10/10], Average Loss: 0.9526, GPU used: 0.59 G
Finished Training
Training time: 221.00 s


# 5. Eval

In [21]:
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the 10000 test images: {100 * correct / total} %')

Accuracy of the model on the 10000 test images: 66.12 %
